In [1]:
%matplotlib inline  
import os
import numpy as np
from PIL import Image
import cv2
from matplotlib.pyplot import figure, imshow, savefig
import matplotlib.pyplot as plt
import collections
import os

In [ ]:
data_dir = 'datasets/bdd/seg/labels/val'

In [ ]:
for jpg in os.listdir(data_dir):
    image = Image.open(os.path.join(data_dir, jpg))
    image = np.array(image)
    print(jpg)
    print(image.shape)
    break

**Histogram of Pedestrian Bounding Box**

In [ ]:
sample_img = 'datasets/bdd/seg/presentation/d0109b85-00000000.png'
sample_img_color = 'datasets/bdd/seg/presentation/d0109b85-00000000_test_color.png'
image = np.array(Image.open(sample_img))
image_color = np.array(Image.open(sample_img_color))

In [ ]:
center_x = 420
center_y = 570
size = 224
tl = [center_x - size / 2, center_y - size / 2]
bl = [center_x - size / 2, center_y + size / 2]
br = [center_x + size / 2, center_y + size / 2]
tr = [center_x + size / 2, center_y - size / 2]
points = np.array([tl, bl, br, tr], dtype=np.int32)

In [ ]:
line_color = (255, 0, 0)
cv2.polylines(image_color, [points], True, line_color, 2)

In [ ]:
figure(figsize=(10, 10))
imshow(image_color)
savefig("bounding_box.png")

In [ ]:
left_x = int(center_x - size / 2)
right_x = int(center_x + size / 2)
top_y = int(center_y - size / 2)
bottom_y = int(center_y + size / 2)

In [ ]:
labels = {
    0: 'road', 
    1: 'sidewalk',
    2: 'building',
    3: 'wall',
    4: 'fence',
    5: 'pole',
    6: 'traffic light',
    7: 'traffic sign',
    8: 'vegetation',
    9: 'terrain',
    10: 'sky',
    11: 'person',
    12: 'rider',
    13: 'car',
    14: 'truck',
    15: 'bus',
    16: 'train',
    17: 'motorcycle',
    18: 'bicycle',
}


In [ ]:
histogram = {}
for key in labels.keys():
    histogram[key] = 0
for y in range(top_y, bottom_y):
    for x in range(left_x, right_x):
        if image[y, x] in histogram:
            histogram[image[y, x]] += 1
names = []
values = []
for key in sorted(histogram.keys()):
    names.append(labels[key])
    values.append(histogram[key])
figure(figsize=(20, 10))
plt.title("Histogram of bounding box")
plt.xlabel("Class Name")
plt.ylabel("Count")
plt.bar(names, values)
savefig("histogram_bounding_box.png")

**Mapillary**

In [ ]:
sample_instance = 'datasets/mapillary/grDzFDON_vntlJOAKJx7qA_instance.png'
sample_label = 'datasets/mapillary/grDzFDON_vntlJOAKJx7qA_label.png'
sample_panoptic = 'datasets/mapillary/grDzFDON_vntlJOAKJx7qA_panoptic.png'
img_instance = np.array(Image.open(sample_instance))
img_label = np.array(Image.open(sample_label))
img_panoptic = np.array(Image.open(sample_panoptic))

What is stored in an instance map: 8 bits of object ids + 8 bits of instance ids

In [2]:
train_images_dir = 'datasets/mapillary/training/images'
train_labels_dir = 'datasets/mapillary/training/train_ids'
val_images_dir = 'datasets/mapillary/validation/images'
val_labels_dir = 'datasets/mapillary/validation/train_ids'
test_images_dir = 'datasets/mapillary/testing/images'
train_images = os.listdir(train_images_dir)
val_images = os.listdir(val_images_dir)

In [ ]:
print("Test training...")
for jpg in train_images:
    image = Image.open(os.path.join(train_images_dir, jpg))
    label = Image.open(os.path.join(train_labels_dir, jpg.split('.')[0] + '.png'))
    if image.width != label.width or image.height != label.height: 
        print("Image:", image.width)
        print("Label:", label.width)

print("Test validation...")
for jpg in val_images:
    image = Image.open(os.path.join(val_images_dir, jpg))
    label = Image.open(os.path.join(val_labels_dir, jpg.split('.')[0] + '.png'))
    if image.width != label.width or image.height != label.height: 
        print("Image:", image.width)
        print("Label:", label.width)
print("Finish test!")

Filter out images that do not have enough crosswalk pixels

After applying threshold, we are able to generate: 

485 crosswalk masks in training. 53 crosswalk masks in validation.

In [16]:
train_labels = os.listdir(train_labels_dir)
val_labels = os.listdir(val_labels_dir)
train_output_dir = 'datasets/mapillary/training/train_ids_color'
val_output_dir = 'datasets/mapillary/validation/train_ids_color'


DUO_PALETTE = np.array([
    [217, 83, 79],
    [0, 0, 0]], dtype=np.uint8)

for filename in train_labels:
    image = np.array(Image.open(os.path.join(train_labels_dir, jpg)))
    output_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    output_image[image == 0] = DUO_PALETTE[0]
    output_image[image == 1] = DUO_PALETTE[1]
    output_image = Image.fromarray(output_image)
    output_path = os.path.join(train_output_dir, filename)
    output_image.save(output_path)

KeyboardInterrupt: 

In [ ]:
for filename in val_labels:
    image = np.array(Image.open(os.path.join(val_labels_dir, jpg)))
    output_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    output_image[image == 0] = DUO_PALETTE[0]
    output_image[image == 1] = DUO_PALETTE[1]
    output_image = Image.fromarray(output_image)
    output_path = os.path.join(val_output_dir, filename)
    output_image.save(output_path)